# EEG - MNIST project.

**Author:** Idriz Pelaj

We use the data provided kindly by the people over at [MindBigData](https://mindbigdata.com/opendb/index.html) in order to train a convolutional neural network on the spectrograms of EEG data so that we can sort of map what MNIST numbers people are looking at and thinking about.

## Installing dependencies

In [ ]:
%pip install torch torchvision numpy Pillow matplotlib requests scipy opencv-python

# Database Helpers
Some functions here will be used as database helpers for our sqlite3 database.

In [ ]:
import numpy as np
import sqlite3
import io

# Helper functions to help with converting numpy arrays to buffers that we can write to the database, and from buffers to arrays that we can load from the database.

"""
Converts a numpy array to bytes.
"""
def numpy_arr_to_bytes(arr):
    buf = io.BytesIO()
    np.save(buf, arr)
    buf.seek(0) # Go back to 0
    return buf.read() # Read the result.

"""
Convert bytes to a numpy array.
"""
def bytes_to_numpy_arr(bytes):
    buf = io.BytesIO(bytes)
    return np.load(buf)

def db_connect():
    return sqlite3.connect('spectrograms.db')

# First Time Setup
This is mostly concerned with setting up things such as the spectrogram database that we will be using to train our neural network.

## Obtaining and preparing the MW data

Firstly, we download the MW data from the official URL

In [ ]:
import requests
import zipfile
import io

url = "https://mindbigdata.com/opendb/MindBigData-MW-v1.0.zip" # We are using the MindWave data - it's less work, and should work as a baseline for testing the model architecture.
response = requests.get(url)
response.raise_for_status() # This will let us assert the request was successful.

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall("mw_data") # Unzips and extracts to our mw_data folder.

Now, we need to actually parse this data. Rather trivial, as the file format is given on the official MindBigData website - and since we're working with MindWave data, this actually becomes a lot easier, too!

In [ ]:
import numpy as np

"""
Reads the data from the MindWave.
Returns a tuple of (code, values)
"""
def read_data(path):
    with open(path, "r") as f:
        rows = f.readlines()
        rows_data = []
        for row in rows:
            code = int(row.split("\t")[4])
            values = np.array([int(x) for x in row.split("\t")[6].split(",")])
            rows_data.append((code, values))
        return rows_data

raw_eeg_data = read_data("./mw_data/MW.txt") # This will give us a tuple array of (code, values).
[x[1].shape for x in raw_eeg_data]

What we can immediately see is that the number of rows for each EEG reading is not exactly the same. We get around this by "forcing" a certain resolution later on, which is not optimal for the spectral characteristics for the signal, but from my tests it didn't seem to have too much of an impact.

## Plotting the signals for clarity
We'll plot a few of the signals we're working with for clarity. It should simply serve as a reminder of what we're working with, and also plots are nice! In the mean time, we'll define some helper functions for the data - namely, finding the first occurrence of a "code".

In [ ]:
"""
Finds the first "code" occurrence in the MW data.
"""
def find_first_code_occurrence(rows, code):
        for row in rows:
            if row[0] == code:
                return row

### Plotting the raw EEG signals

In [ ]:
from matplotlib import pyplot as plt

# This will plot all the data necessary.
for l in range(-1, 9):
        code, data = find_first_code_occurrence(raw_eeg_data, l)
        t = np.arange(0, len(data))

        plt.plot(t, data, label=f"Number: {code}")
        plt.xlabel("n")
        plt.ylabel("raw_eeg_data[n] amplitude(in µV)")
        plt.ylim([-250, 500])
        plt.legend()

It looks quite noisy! The amplitude is in $\mu V$, and we index by $n$.

### Plotting the Power Spectral Density(PSD) of these signals

For fun, we can also see the PSD of these signals - what we'll notice is that the average frequency range(in CT) fluctuates in an interval around ±200Hz. The sampling frequency $f_s$ is given at around ~512Hz, which means our nyquist frequency is around ~256Hz.

In [ ]:
from scipy.signal import welch

f_s = 512 # The MindWave samples at around 512Hz.

for l in range(-1, 9):
    code, data = find_first_code_occurrence(raw_eeg_data, l)
    frequencies, psd = welch(data, f_s, nperseg=1024) # We compute the FFT using welch's algorithm.

    # Plot the PSD
    plt.semilogy(frequencies, psd)
    plt.grid()
    plt.title('Power Spectral Density')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')

This is the power spectral density as obtained using [Welch's algorithm](https://en.wikipedia.org/wiki/Welch%27s_method).

### Plotting the spectrogram of a signal.
Here, we'll plot the spectrogram of _one_ signal. 

In [ ]:
from scipy.signal import ShortTimeFFT
def plot_spectrogram(data, code, f_s): # A helper method to plot the spectrogram.
    data = np.array(data)

    win = ('gaussian', 1e-2 * f_s) # A gaussian window should be fine.
    N = len(data)
    
    t = np.arange(N) / f_s
    nperseg, noverlap = 50, 40 # These parameters seem to work fine in obtaining a good spectrogram.

    SFT = ShortTimeFFT.from_window(win, f_s, nperseg, noverlap, fft_mode='centered', scale_to='magnitude', phase_shift=None)

    Sz1 = SFT.stft(data) 
    plt.figure()

    t_lo, t_hi, f_lo, f_hi = SFT.extent(N, center_bins=True)

    plt.ylim(f_lo, f_hi)
    plt.xlim(t_lo, t_hi)
    plt.title(f"Number: {code}")
    plt.xlabel(rf"Time $t$ in seconds ($\Delta t= {SFT.delta_t:g}\,$s)")

    extent = SFT.extent(N, center_bins=True)

    kw = dict(origin='lower', aspect='auto', cmap='viridis')
    im1b = plt.imshow(abs(Sz1), extent=extent, **kw)
    plt.colorbar(im1b, label="Magnitude $|S_z(t, f)|$")
    _ = plt.ylabel(r"Frequency $f$ in Hertz ($\Delta f = %g\,$Hz)" %
                   SFT.delta_f, x=0.08, y=0.5, fontsize='medium')

first_elem_code, first_elem_data = find_first_code_occurrence(raw_eeg_data, 1)
plot_spectrogram(first_elem_data, 0, f_s)

Pretty cool, no? The MindBigData website also does say that the duration is around 2 seconds, so we further verify that we have done everything correctly so far! Of course - do recall this is the amplitude representation. Phase isn't really necessary in my experience, but if it is, please be sure to let me know!

## Preparing the spectrograms of all the data and saving to a SQLite database.
We are now going to prepare the spectrograms of all of the data we have, reshape it - and store it as numpy blobs in our database. This will make it easier to make this data a little more portable, or in our case, not flood our filesystem with npy files!!!

### Inserting the data into the database

We now need to open a database and create the necessary table for our data.

In [ ]:
# Open and create the necessary database and table.
with db_connect() as conn:
    cursor = conn.cursor()

    # Create the table.
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS spectrograms (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        code INTEGER,
        array BLOB NOT NULL
    )
    ''')
    conn.commit()

Now, we obtain all the spectrograms of our data and insert them into the database. We have a helper function for this.

In [ ]:
"""
Returns the spectrogram of the raw EEG data given.
Returns: (Sz1, (t_lo, t_hi), (f_lo, f_hi), code)
"""
def get_spectrogram(data, code, f_s):
    data = np.array(data)

    win = ('gaussian', 1e-2 * f_s)
    N = len(data)

    nperseg, noverlap = 50, 40
    SFT = ShortTimeFFT.from_window(win, f_s, nperseg, noverlap, fft_mode='centered', scale_to='magnitude', phase_shift=None)

    Sz1 = SFT.stft(data)

    t_lo, t_hi, f_lo, f_hi = SFT.extent(N, center_bins=True)
    return (Sz1, (t_lo, t_hi), (f_lo, f_hi), code)

# This will pad all data so that there's 110 columns. This should be fine.
def pad_to_110_cols(spectrogram_data):
    cols_to_pad = 110 - spectrogram_data.shape[1]
    # Use np.pad to add zeros to the right side of the array
    padded_array = np.pad(spectrogram_data, ((0, 0), (0, cols_to_pad)), mode='constant')
    return padded_array

with db_connect() as conn:
    cursor = conn.cursor()
    for code, data in raw_eeg_data:
        amp_arr, (t_lo, t_hi), (f_lo, f_hi), code  = get_spectrogram(data, code, f_s) # Obtains the spectrograms. This will take a bit to compute.
        amp_arr = np.abs(amp_arr).round().astype(np.uint16) # We're working with power magnitude - no negatives, and we should probably stick to integer values, so we can save on some space.
        amp_arr = pad_to_110_cols(amp_arr) # Now we have 110 columns. We are always ensured this is the maximum number of columns, as well as knowing that the maximum number of rows we have is 50.
        bytes = numpy_arr_to_bytes(amp_arr)
        cursor.execute('INSERT INTO spectrograms(array, code) VALUES (?, ?)', (bytes, code))
    conn.commit()
    print(f"Database updated.")


# Neural Network
The main idea behind the neural network is the use of a convolutional neural network. We'll be using PyTorch for it. Initially, this project was built on my own neural network library that I spun up in order to learn more about neural networks, which will be found and referenced on my github later on - however it is far too un-optimized for a project that does this much training.

## Loading the data from the spectrogram database
In order to get started, we must first extract the data from the spectrogram database.

In [ ]:
import sqlite3

# Fetches the spectrograms from the database, pretty straightforward.
def fetch_spectrograms(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    query = "SELECT array, code FROM spectrograms"
    cursor.execute(query)
    rows = cursor.fetchall()
    conn.close()
    
    return [[bytes_to_numpy_arr(array), code] for array, code in rows]

spectrograms_data = fetch_spectrograms("spectrograms.db")
print("Number of entries:", len(spectrograms_data))
print("Example spectrogram entry shape:", spectrograms_data[0][0].shape)


## Normalizing the data
We should normalize the amplitudes first, row-by-row.

In [ ]:

for i in range(0, len(spectrograms_data)):
    data = spectrograms_data[i][0]
    spectrograms_data[i][0] = data / np.max(data) # Normalize row-by-row. 

print("Maximum value of first row:", np.max(spectrograms_data[0][0]))
print("Minimum value of first row:", np.min(spectrograms_data[0][0]))
print("Shape of first row:", spectrograms_data[0][0].shape)

## Building the model
The model is a fairly standard convolutional neural network - we'll slowly learn more and more features of the "images"(which are actually arrays that have been normalized from 0 to 1, with only one channel). We'll build it using pytorch.

Let's update the PyTorch device.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Get cpu, gpu or mps device for training. This was taken directly from pytorch.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

torch.device(device)

Secondly, the testing and train split methods.

In [ ]:
def train_test_split(data, train_percentile=0.8, batch_size=32):
    count = int(len(data) * train_percentile)
    train = data[:count]
    testing = data[count:]

    x_train = torch.tensor([x[0] for x in train], dtype=torch.float32, device=device).unsqueeze(1)
    print(x_train.shape)
    y_train = torch.tensor([x[1] + 1 for x in train], dtype=torch.long, device=device) # we don't want negative indices

    train_data = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    x_test = torch.tensor([x[0] for x in testing], dtype=torch.float32, device=device).unsqueeze(1)
    y_test = torch.tensor([x[1] + 1 for x in testing], dtype=torch.long, device=device) # we don't want negative indices

    test_data = TensorDataset(x_test, y_test)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

    return train_loader, test_loader

train_loader, test_loader = train_test_split(spectrograms_data)

### The CNN
We'll define a pretty conventional convolutional neural network model that has three convolutional layers(as well as does subsampling in the meanwhile), and eventually maps it to 11 categories.

In [ ]:
class EEGCNN(nn.Module):
    def __init__(self):
        super(EEGCNN, self).__init__()
        self.conv_stack = nn.Sequential(
            # First convolutional step
            nn.Conv2d(1, 32, 3), # (48, 108)
            nn.MaxPool2d(2, 2), # Subsample (48, 108) --> (24, 54)
            nn.ReLU(), # Apply non-linearity
            
            # Second convolutional step
            nn.Conv2d(32, 64, 3), # (24, 54)
            nn.MaxPool2d(2, 2), # (24, 54) --> (11, 26)
            nn.ReLU(),

            # Third convolutional step
            nn.Conv2d(64, 128, 3), # (11, 26)
            nn.MaxPool2d(2, 2), # (11, 26) --> (4, 12)
            nn.ReLU(),

            nn.Flatten(), # (N, 128 * 4 * 12)

            nn.Linear(128 * 4 * 12, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 11)
        )

    def forward(self, x):
        logits = self.conv_stack(x)
        return logits

model = EEGCNN().to(device)

# Test out the shape of the output of the model.
test_input = torch.randn(32, 1, 50, 110).type(torch.float32).to(device)
model(test_input).shape

### Training the model

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer): # Taken from the PyTorch docs.
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)

print("Done!")

### Saving the model
We'll save the model.

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")